In [1]:
import pandas as pd
import glob
from pyproj import Proj, transform

In [2]:
# import bike accident data
bike_df = pd.read_excel('data/Final_Bike_Collision_Database.xlsx')

# convert the bike data locations into latitude and longitudes and find the geographic extent
inProj = Proj('epsg:26986')
outProj = Proj('epsg:4326')
xmax,ymax = bike_df['Xkm'].max(),bike_df['Ykm'].max()
xmax2,ymax2 = transform(inProj,outProj,xmax,ymax)
print ('Upper Right: ')
print (xmax2,ymax2)

xmin,ymin = bike_df['Xkm'].min(),bike_df['Ykm'].min()
xmin2,ymin2 = transform(inProj,outProj,xmin,ymin)

print('Bottom Right:')
print(xmin2, ymax2)

print('Top Right: ')
print(xmax2, ymin2)

print ('Bottom Left: ')
print (xmin2,ymin2)

Upper Right: 
42.385920811282396 -71.00003640589748
Bottom Right:
42.23530626734878 -71.00003640589748
Top Right: 
42.385920811282396 -71.17443656586595
Bottom Left: 
42.23530626734878 -71.17443656586595


In [3]:
bike_df.columns

Index(['ID', 'YEAR', 'DATE', 'DAY_WEEK', 'TIME', 'TYPE', 'SOURCE', 'XFINAL',
       'Xkm', 'YFINAL', 'Ykm', 'Address', 'Main', 'RoadType', 'ISINTERSEC',
       'TRACT', 'CouncilDIS', 'Councillor', 'PlanningDi', 'OIF1', 'OIF2',
       'OIF3', 'OIF4', 'BLFinal', 'CS', 'LIGHTING', 'Indoor', 'Light',
       'LightEng', 'WEATHER', 'PrecipCond', 'AtmosCondi', 'DayNight', 'Tmax',
       'Tmin', 'Tavg', 'Temprange', 'SunriseTim', 'SunsetTime', 'SnowFall',
       'PrecipTota', 'Fault', 'Doored', 'HelmetDocu', 'TaxiFinal',
       'hitrunfina', 'AlcoholFin', 'INJURED', 'TRANSPORTE', 'TREATED',
       'GENDER', 'ETHNICITY', 'AGE', 'Narrative'],
      dtype='object')

I found https://gis.massdot.state.ma.us/roadinventory/ which had translated a lot of the columns in the 2018 DOT database, which was helpful
I downloaded 7 files that should cover the area I'm interested in for Bike Accidents

In [4]:
boston_roads = pd.DataFrame()
for file in glob.glob('data/Road Inventory*.csv'):
    df = pd.read_csv(file, dtype={'Pedestrian Surface Type': 'str', 
                                  'Pedestrian Facility Type': 'str', 
                                  'Truck Network': 'str',
                                  'Shldr_UL_T': 'str',
                                  'StationID': 'str'})
    boston_roads = boston_roads.append(df)



In [5]:
# drop duplicates

boston_roads.drop_duplicates(inplace=True, subset='OBJECTID')

In [6]:
# print out road columns

boston_roads.columns

Index(['OBJECTID', 'NHS', 'Jurisdictn', 'Functional Class',
       'Federal Functional Class', 'Federal Aid Eligible Road',
       'Facility Type', 'Urbanized Area', 'Urban Type', 'Access Control',
       'Number of Travel Lanes', 'Opposite Number of Travel Lanes',
       'Surface Type', 'Surface Width', 'Shoulder Width Right', 'Shldr_Rt_T',
       'Shoulder Width Left', 'Shldr_Lt_T', 'Shoulder Width Left.1',
       'Shldr_UL_T', 'Median Width', 'Med_Type', 'Curb', 'Left Sidewalk Width',
       'Right Sidewalk Width', 'Pedestrian Surface Type',
       'Pedestrian Facility Type', 'Cross Slope', 'Off-road Path Width',
       'Street Operation', 'Speed Limit', 'Speed Limit Opposing Direction',
       'Speed Limit.1', 'Truck Exclusion Type', 'Truck Exclusion Time',
       'Truck Permit', 'Truck Network', 'Truck Route', 'Right Of Way Width',
       'Structural Condition', 'Terrain Type', 'Toll Road',
       'Metropolitian Highway System', 'City', 'Municipal Type', 'County',
       'Highway 

In [7]:
# some exploration to understand column values better

column_list = list(boston_roads.columns)

# remove columns where it doesn't make sense to look at the list of unique values
vals_to_remove = ['OBJECTID', 'x', 'y', 'Truck Permit', 'Station Number', 'Street Name', 'From Street Name',
                 'To Street Name', 'AADT', 'StationID', 'Length', 'Route ID', 'From Measure', 'To Measure']
for val in vals_to_remove:
    column_list.remove(val)
for col in column_list:
    print(col)
    print(boston_roads[col].unique())

NHS
['Not on NHS' nan 'NHS - Interstate'
 'NHS - Major Public Transit or Multi-Modal Passenger Terminal'
 'NHS - Principal Arterial (MAP-21)'
 'NHS - Other (not in above categories)'
 'NHS - Other truck route exclusion' 'NHS - STRAHNET Connector'
 'NHS - Major Airport' 'NHS - Other - One-way pair'
 'NHS - Major Amtrak Station' 'NHS - Major Port Facility'
 'NHS - Major Rail/Truck terminal' 'NHS - Major Intercity Bus Terminal']
Jurisdictn
['Unaccepted by city or town' 'City or Town accepted road' nan
 'Massachusetts Department of Transportation'
 'State college or university' 'Department of Conservation and Recreation'
 'Private' 'Massachusetts Port Authority' 'Federal Institutional'
 'State Institutional' 'US Army Corps of Engineers']
Functional Class
['Local' nan 'Interstate'
 'Rural minor arterial or urban principal arterial'
 'Urban minor arterial or rural major collector'
 'Rural or urban principal arterial'
 'Urban collector or rural minor collector']
Federal Functional Class
['Loc

In [11]:
car_acc_df = pd.read_csv('data/2019_Crashes_.csv', dtype={'DIST_DIRC_EXIT': 'str', 'FD_AID_RTE': 'str', 'URBAN_LOC_TYPE': 'str'})

In [12]:
for col in car_acc_df.columns:
    print(col)

OBJECTID
CRASH_NUMB
CITY_TOWN_NAME
CRASH_DATE
CRASH_HOUR
CRASH_TIME_2
CRASH_STATUS
CRASH_SEVERITY_DESCR
MAX_INJR_SVRTY_CL
NUMB_VEHC
NUMB_NONFATAL_INJR
NUMB_FATAL_INJR
POLC_AGNCY_TYPE_DESCR
MANR_COLL_DESCR
VEHC_MNVR_ACTN_CL
VEHC_TRVL_DIRC_CL
VEHC_SEQ_EVENTS_CL
AMBNT_LIGHT_DESCR
WEATH_COND_DESCR
ROAD_SURF_COND_DESCR
FIRST_HRMF_EVENT_DESCR
MOST_HRMFL_EVT_CL
DRVR_CNTRB_CIRC_CL
VEHC_CONFIG_CL
STREET_NUMB
RDWY
DIST_DIRC_FROM_INT
NEAR_INT_RDWY
MM_RTE
DIST_DIRC_MILEMARKER
MILEMARKER
EXIT_RTE
DIST_DIRC_EXIT
EXIT_NUMB
DIST_DIRC_LANDMARK
LANDMARK
RDWY_JNCT_TYPE_DESCR
TRAF_CNTRL_DEVC_TYPE_DESCR
TRAFY_DESCR_DESCR
JURISDICTN
FIRST_HRMF_EVENT_LOC_DESCR
NON_MTRST_TYPE_CL
NON_MTRST_ACTN_CL
NON_MTRST_LOC_CL
IS_GEOCODED
GEOCODING_METHOD_NAME
X
Y
LAT
LON
RMV_DOC_IDS
CRASH_RPT_IDS
YEAR
AGE_DRVR_YNGST
AGE_DRVR_OLDEST
AGE_NONMTRST_YNGST
AGE_NONMTRST_OLDEST
DRVR_DISTRACTED_CL
DISTRICT_NUM
RPA_ABBR
VEHC_EMER_USE_CL
VEHC_TOWED_FROM_SCENE_CL
CNTY_NAME
FMCSA_RPTBL_CL
FMCSA_RPTBL
HIT_RUN_DESCR
LCLTY_NAME
ROAD_CNTR